## Historical Factor Scenario

In a historical factor scenario, the aim is to replicate the market conditions during a significant or
topical historical event and estimate the impact on your portfolio today. The first step is to define the time period
 of the historical event. Then, factor returns across the time period are geometrically aggregated, and these aggregated
 returns are the factor shocks that will be used to derive the estimated impact on the performance of your portfolio.

### Step 1: Initialize a GsSession

The initial step is to import all required modules and initialize a GsSession with your application ID and secret.

In [ ]:
from gs_quant.session import GsSession, Environment
from gs_quant.markets.scenario import FactorScenario, FactorScenarioType, HistoricalSimulationParameters
from gs_quant.entities.entity import ScenarioCalculationMeasure
from gs_quant.markets.portfolio_manager import PortfolioManager
from gs_quant.entities.entitlements import Entitlements, EntitlementBlock, User
import datetime as dt

client = None
secret = None

## External users must fill in their client ID and secret below and comment out the lines below

# client = 'ENTER CLIENT ID'
# secret = 'ENTER CLIENT SECRET'

GsSession.use(Environment.PROD, client_id=client, client_secret=secret)

print('GS Session initialized.')

### Step 2: How do I create a historical factor scenario?

Similar to factor shock scenarios, we will need to provide:
- A name for the scenario
- The type of the scenario. In this case, the scenario type is `FactorScenarioType.Factor_Historical_Simulation`
- The parameters of the scenario
- Scenario entitlements
- A description for the scenario

The parameters are defined in a `HistoricalSimulationParameters` object and will define the time
period of the historical simulation scenario:

- Start Date: The start date of the historical event simulation period
- End Date: The end date of the historical event simulation period.


In [ ]:
start_date = dt.date(2023, 1, 1)
end_date = dt.date(2024, 1, 1)

admin_emails = ["ADMIN EMAILS"]
edit_emails = ["EDIT EMAILS"]
view_emails = ["VIEW EMAILS"]

admin_entitlements = EntitlementBlock(users=User.get_many(emails=admin_emails))
edit_entitlements = EntitlementBlock(users=User.get_many(emails=edit_emails))
view_entitlements = EntitlementBlock(users=User.get_many(emails=view_emails))

scenario_entitlements = Entitlements(view=view_entitlements, edit=edit_entitlements, admin=admin_entitlements)

date_range_parameters = HistoricalSimulationParameters(start_date=start_date, end_date=end_date)
historical_scenario_name = (
    f"Example historical simulation scenario from {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}"
)
historical_replay_scenario = FactorScenario(
    name=historical_scenario_name,
    type=FactorScenarioType.Factor_Historical_Simulation,
    parameters=date_range_parameters,
    description=f"Sample historical simulation scenario from {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}",
    entitlements=scenario_entitlements,
)

historical_replay_scenario.save()
print(historical_replay_scenario)

## FAQs:

### How do I get my Scenarios or Scenarios that are shared with me?

You can get saved scenarios that you created or that were shared with you:
- By its unique Marquee ID
- By its name
- By applying filters such as `risk_model`, `type` of the scenario, the `shocked_factors`, or whether the shocks are propagated
- For a historical simulation scenario, you can filter by a `start_date` and `end_date`.

In [ ]:
risk_model_id = "RISK MODEL ID"

# Get by id
scenario_by_id = FactorScenario.get("SCENARIO ID")

# Get all my factor Shock scenarios (and those that are shared with me) with risk model AXIOMA_AXWW4M and whose
# factor shocks are propagated to all factors
many_scenarios = FactorScenario.get_many(
    risk_model=risk_model_id, propagated_shocks=True, type=FactorScenarioType.Factor_Shock
)
# Get all my factor Shock scenarios (and those that are shared with me) with risk model AXIOMA_AXWW4M that shocks the
# Value factor
many_scenarios = FactorScenario.get_many(
    risk_model=risk_model_id, shocked_factors=["Value"], type=FactorScenarioType.Factor_Shock
)

# Get all my historical simulation scenarios that are within the requested start and end date
historical_simulation_scenarios = FactorScenario.get_many(
    type="Factor Historical Simulation", start_date=dt.date(2023, 1, 1), end_date=dt.date(2024, 1, 1)
)

### How do I get GS pre-scanned scenarios?

To get GS pre-scanned scenarios, you can additionally filter by the tag "GS". See below for some examples

In [ ]:
# Get 3 factor shocks scenarios with risk model AXIOMA_AXWW4M that propagate factor shocks
pre_canned_factor_shock_scenarios = FactorScenario.get_many(
    risk_model=risk_model_id, propagated_shocks=True, type=FactorScenarioType.Factor_Shock, tags=["GS"], limit=3
)
print(pre_canned_factor_shock_scenarios)

# Get 3 historical simulation that are within selected start and end date
pre_canned_historical_simulation_scenarios = FactorScenario.get_many(
    type="Factor Historical Simulation",
    start_date=dt.date(2010, 1, 1),
    end_date=dt.date(2024, 1, 1),
    tags=["GS"],
    limit=3,
)
print(pre_canned_historical_simulation_scenarios)

### Running Scenario(s) on a Portfolio

To see the estimated impact of scenarios on your portfolio, run the `get_factor_scenario_analytics` function in the
`PortfolioManager` class, which is the interface for managing your portfolios in Marquee.

A scenario calculation request requires the following components:

- A date: date on which to run a scenario. We will run the scenario(s) on your portfolio holdings as of this date.
- Scenarios: List of scenarios to run. Can either be a list of unique Marquee IDs, or a scenario object itself.
- Risk Model: The risk model to run a scenario with.
- Measures: Metrics to return, which include:
    1. Total portfolio estimated factor PnL
    2. Total portfolio PnL by GICS sector & industry
    3. Total portfolio PnL by region
    4. Total portfolio PnL by the direction (LONG/SHORT) of your portfolio holdings,
    5. Estimated performance of each asset in your portfolio.

In [ ]:
portfolio_id = "PORTFOLIO ID"
pm = PortfolioManager(portfolio_id)

scenario_analytics = pm.get_factor_scenario_analytics(
    scenarios=["SCENARIO ID"],
    date=dt.date(2024, 4, 10),
    measures=[
        ScenarioCalculationMeasure.SUMMARY,
        ScenarioCalculationMeasure.ESTIMATED_FACTOR_PNL,
        ScenarioCalculationMeasure.ESTIMATED_PNL_BY_SECTOR,
        ScenarioCalculationMeasure.ESTIMATED_PNL_BY_REGION,
        ScenarioCalculationMeasure.ESTIMATED_PNL_BY_DIRECTION,
        ScenarioCalculationMeasure.ESTIMATED_PNL_BY_ASSET,
    ],
    risk_model=risk_model_id,
)
summary = scenario_analytics.get('summary')
factor_pnl = scenario_analytics.get('factorPnl')
sector_aggregations = scenario_analytics.get('bySectorAggregations')
region_aggregations = scenario_analytics.get("byRegionAggregations")
by_asset_aggregations = scenario_analytics.get('byAsset')

### Run Scenario(s) on a Basket

We can also run one or multiple scenarios on an existing equity custom basket and pull scenario analytics data.

In [ ]:
from gs_quant.markets.baskets import Basket

basket_ticker = "BASKET TICKER"
basket = Basket.get(basket_ticker)

basket_scenario_analytics = basket.get_factor_scenario_analytics(
    scenarios=["YOUR SCENARIOS"],
    date=dt.date(2024, 4, 1),
    measures=[
        ScenarioCalculationMeasure.SUMMARY,
        ScenarioCalculationMeasure.ESTIMATED_FACTOR_PNL,
        ScenarioCalculationMeasure.ESTIMATED_PNL_BY_SECTOR,
        ScenarioCalculationMeasure.ESTIMATED_PNL_BY_REGION,
        ScenarioCalculationMeasure.ESTIMATED_PNL_BY_DIRECTION,
        ScenarioCalculationMeasure.ESTIMATED_PNL_BY_ASSET,
    ],
    risk_model=risk_model_id,
)

summary = basket_scenario_analytics.get('summary')
factor_pnl = basket_scenario_analytics.get('factorPnl')
sector_aggregations = basket_scenario_analytics.get('bySectorAggregations')
region_aggregations = basket_scenario_analytics.get("byRegionAggregations")
by_asset_aggregations = basket_scenario_analytics.get('byAsset')